In [ ]:
##############      Configuración      ##############
import sys
sys.path.append('..')  # Add the parent directory to the sys.path
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict
from dotenv import dotenv_values

pd.set_option("display.max_columns", None)
env = dotenv_values("/mnt/d/Maestría/Tesis/Repo/scripts/globals.env")

path_proyecto = env["PATH_PROYECTO"]
path_datain = env["PATH_DATAIN"]
path_dataout = env["PATH_DATAOUT"]
path_out_imgs = r"/mnt/e/Tesis Maestría/imagenes"
path_scripts = env["PATH_SCRIPTS"]
path_satelites = env["PATH_SATELITES"]
path_logs = env["PATH_LOGS"]
path_outputs = env["PATH_OUTPUTS"]

In [2]:
import earthpy.plot as ep
import build_dataset as bd
import utils
import geopandas as gpd
import shapely
import skimage
import folium
import pandas as pd
import seaborn as sns

raster_size = 58.435144 - 58.435139 # In epsg 4326 degrees

In [ ]:
datasets, extents = bd.load_satellite_datasets()
icpag = bd.load_icpag_dataset()
icpag = bd.assign_links_to_datasets(icpag, extents)

# Imagenes de Ejemplo

In [ ]:
import random
import skimage
import cv2
import os
img_folders = os.listdir(path_out_imgs)
img_folders = [folder for folder in img_folders if "train" in folder]

fig, axs = plt.subplots(4,7, figsize=(5*7, 5*4))
for row, folder in enumerate(img_folders):

    imgs = os.listdir(f"{path_out_imgs}/{folder}")
    show_imgs = random.sample(imgs, 7)

    for col, im_path in enumerate(show_imgs):

        im = np.load(f"{path_out_imgs}/{folder}/{im_path}")
        im = np.moveaxis(im, 0, 2)
        im = im[:,:,:3]
        # Equalize hist
        im = skimage.exposure.equalize_hist(im)
        im = cv2.resize(im, (128, 128))

        axs[row][col].imshow(im[:,:,:3])
        axs[row][col].set_axis_off()
        axs[row][col].set_title(f"{folder}_{row}")
        
# Remove axis lables
plt.axis("tight")

# Create training/validaton_by_epoch

In [3]:
path_repo = r"/mnt/d/Maestría/Tesis/Repo"
# path_repo = r"D:/Maestría/Tesis/Repo"
path_outputs = f"{path_repo}/outputs"
path_dataout = f"{path_repo}/data/data_out"


In [4]:
models =[
    "mobnet_v3_size128_tiles1_sample5",
    # "mobnet_v3_size256_tiles1_sample1",
    # "mobnet_v3_size512_tiles1_sample1",
    # "mobnet_v3_size128_tiles2_sample20",
]

old_models = { 
        'mobnet_v3_size128_tiles1_sample1':
            [
                'mobnet_v3_20230908-203439',
                'mobnet_v3_20230909-020833',
                'mobnet_v3_20230909-121048',
                'mobnet_v3_20230909-153231',
                'mobnet_v3_20230909-202517',
                'mobnet_v3_20230909-213040',
                'mobnet_v3_20230910-130103',
                'mobnet_v3_20230911-092415',
                'mobnet_v3_20230911-231611',
                'mobnet_v3_20230912-012855',
            ],
        'mobnet_v3_size256_tiles1_sample1':
            [       
                'mobnet_v3_20230914-215006',
                'mobnet_v3_20230913-200010',
            ],      
        'mobnet_v3_size512_tiles1_sample1':
            [
                'mobnet_v3_20230915-122541',
                'mobnet_v3_20230916-124818',
            ],
        'mobnet_v3_size128_tiles2_sample20':
            [
                'mobnet_v3_20230918-192554',
                'mobnet_v3_20230918-231134',
                'mobnet_v3_20230919-104912',
                'mobnet_v3_20230919-234505',
                'mobnet_v3_20230920-131123',
                'mobnet_v3_20230920-180458',
                'mobnet_v3_20230920-194725',
            ]
    }

In [ ]:
import tensorflow as tf
import tensorboard as tb
import numpy as np

# First, upload experiment to Tensorboard:
# 
# Extract data from TB
experiment_id = "CwPHXf5GQ9qdvTQcZoNyow"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()

for kind in ["train", "validation"]:
    # Keep only train values
    base = df[df.run.str.contains(f'{kind}') & (df.tag == 'epoch_loss')]
    base['run'] = base.run.str.replace(f"/{kind}", "")

    # Filter model
    models_by_date = {
        'mobnet_v3_size128_tiles1_sample5':[""]
    }

    base['model'] = np.nan
    for model, names in models_by_date.items():
        for name in names:
            base.loc[base.run.str.contains(name), 'model'] = model
            
    # Reshape data
    plot_data = base[~base.model.isna()]\
        .drop_duplicates(subset=['model','step'], keep='first')\
        .pivot(index='step', columns='model', values='value')
        
    # Export data
    plot_data.to_csv(f"{path_repo}/data/data_out/{kind}_by_epoch.csv")

# True Mean Squared error

In [10]:
from tqdm import tqdm
import pandas as pd

path_repo = r"/mnt/d/Maestría/Tesis/Repo"
# path_repo = r"D:/Maestría/Tesis/Repo"
path_outputs = f"{path_repo}/outputs"
path_dataout = f"{path_repo}/data/data_out"

def get_model_results(modelname):
    
    folder = f"{path_repo}/data/data_out/models_by_epoch/{modelname}"
 
    # Get train dataset
    train_data = pd.read_csv(rf"{path_dataout}/train_by_epoch.csv")
    mse_train_df = train_data[[modelname]]
    # TODO: Calcular el R2
    mse_train_df = mse_train_df.reset_index().rename(columns={'index':'epoch', modelname:'mse_train'})

    # Get validation (test by each image) dataset
    test2_data = pd.read_csv(rf"{path_dataout}/validation_by_epoch.csv")
    mse_test2_df = test2_data[[modelname]]
    # TODO: Calcular el R2
    mse_test2_df = mse_test2_df.reset_index().rename(columns={'index':'epoch', modelname:'mse_test_imgs'})

    # Get test dataset
    mse_test_df = pd.DataFrame()
    for epoch in tqdm(range(0, 200)):

        df = pd.read_csv(f"{folder}/{modelname}_{epoch}.csv")
        mse = df.groupby("link").sq_error.mean().mean()

        mse_test_df.at[epoch, 'mse_test_rc'] = mse
        mse_test_df.at[epoch, 'epoch'] = epoch    
    
    # Merge both df
    mse_df = mse_train_df.merge(mse_test_df, on='epoch')
    mse_df = mse_df.merge(mse_test2_df, on='epoch')

    return mse_df

In [ ]:
def plot_mse_over_epochs(mse_df, modelname, metric="mse", save=False):
    import plotly.express as px
    from plotly import graph_objects as go

    plot_df = mse_df.melt(id_vars='epoch', value_vars=['mse_test_imgs', 'mse_test_rc','mse_train'])

    # Plot
    fig = px.line(plot_df, x="epoch", y="value", color="variable", title='True Mean Squared Error over epochs')
    fig.update_yaxes(range=[0, 1])

    fig.update_layout(
        autosize=False,
        width=1280,
        height=720,)

    if save:
        fig.write_image(f"{path_outputs}/mse_best_prediction_{modelname}.png")

    return fig
    
def plot_predictions_vs_real(mse_df, modelname, quantiles=False, last_training=False, save=False):
    import plotly.express as px
    from plotly import graph_objects as go

    folder = f"{path_repo}/data/data_out/models_by_epoch/{modelname}"
    
    # Select best epoch... ¿Is this correct?       
    best_case_epoch = mse_df.loc[mse_df["mse_test_rc"]==mse_df["mse_test_rc"].min()].index.values[0]

    if last_training:
        best_case_epoch = 199
        
    # Open dataset
    best_case = pd.read_csv(
        rf"{folder}/{modelname}_{best_case_epoch}.csv"
        )
    best_case = best_case.groupby("link")[['real_value', 'mean_prediction']].mean().reset_index()
    if quantiles:
        best_case['real_value'] = pd.qcut(best_case['real_value'], 100, labels=False)
        best_case['mean_prediction'] = pd.qcut(best_case['mean_prediction'], 100, labels=False)
        axis_range = [0, 100]
        title = f"{modelname} - cuantiles"
    else:
        axis_range = [-2, 2]
        title = f"{modelname} - niveles"
        
    import seaborn as sns
    fig = px.scatter(best_case, x="real_value", y="mean_prediction", hover_data=["link"],
                     title=title)
    fig.update_yaxes(range=axis_range)
    fig.update_xaxes(range=axis_range)
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,)

    # Add 45° line
    line_fig = go.Figure(data=go.Scatter(x=best_case['real_value'], y=best_case['real_value'], mode='lines', name='45°'))
    fig.add_trace(line_fig.data[0])

    if save:
        if quantiles:
            fig.write_image(f"{path_outputs}/prediction_vs_real_best_prediction_{modelname}_q.png")
        else:
            fig.write_image(f"{path_outputs}/prediction_vs_real_best_prediction_{modelname}.png")
    return fig

In [12]:
def plot_all_over_epochs(modelnames, metric="mse", save=True):
    import plotly.express as px
    from plotly import graph_objects as go

    plot_data = []
    for modelname in modelnames:
        # Get data
        mse_df = get_model_results(modelname)

        # Prepare dataset for plot
        var_x = icpag.groupby("link")['var'].first().var()
        mse_df['r2'] = 1 - mse_df.mse / var_x
        mse_df['smoothed_mse'] = mse_df.rolling(20).mse.mean()
        mse_df['smoothed_r2'] = mse_df.rolling(20).r2.mean()

        melt_df = mse_df.melt(id_vars='epoch')
        plot_df = melt_df[melt_df.variable.isin([metric, f'smoothed_{metric}'])] 
        plot_df['name'] = f"{modelname}" + plot_df["variable"]
        plot_data += [plot_df.copy()]
        
    plot_data = pd.concat(plot_data)
    # Plot
    fig = px.line(plot_data, x="epoch", y="value", color="name")

    if metric=="mse":
        optimal_value_y = mse_df["mse"].min()
        name = "min_mse"
    elif metric=="r2":
        optimal_value_y = mse_df["r2"].max()
        name = "max_r2"
    else:
        raise ValueError("metric has to be either 'mse' or 'r2'")
    
    fig.update_layout(
        autosize=False,
        width=1200,
        height=800,)

    
    fig.update_yaxes(range=[0.1, 1.2])
        
    if save:
        main_fig.write_image(f"{path_outputs}/{metric}_best_prediction_{modelname}.png")

    # fig.write_image(f"{path_outputs}/mse_best_prediction_mobnet_v3_size{size}_tiles{tiles}_sample{sample}.png")
    return fig


In [13]:
models =[
    # "mobnet_v3_size128_tiles1_sample1",
    # "mobnet_v3_size256_tiles1_sample1",
    # "mobnet_v3_size512_tiles1_sample1",
    "mobnet_v3_size128_tiles1_sample5",
]

for modelname in models:
    mse_df = get_model_results(modelname)
    plot_mse_over_epochs(mse_df, modelname, metric="mse", save=True)
    plot_mse_over_epochs(mse_df, modelname, metric="r2", save=True)
    plot_predictions_vs_real(mse_df, modelname=modelname, save=True)
    plot_predictions_vs_real(mse_df, modelname=modelname, quantiles=True, save=True)


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:12<00:00, 15.72it/s]


In [9]:
modelname = rf"mobnet_v3_size128_tiles1_sample1"
mse_df = get_model_results(modelname)

KeyError: "None of [Index(['mobnet_v3_size128_tiles1_sample1'], dtype='object')] are in the [columns]"

In [ ]:
plot_predictions_vs_real(mse_df, modelname=modelname)

In [ ]:
plot_predictions_vs_real(mse_df, modelname=modelname, quantiles=True)

# Gridded predictions

In [ ]:
import os
import importlib
import build_dataset
import run_model
import tensorflow as tf
importlib.reload(build_dataset)
importlib.reload(run_model)

# Load data
datasets, extents = build_dataset.load_satellite_datasets()
icpag = build_dataset.load_icpag_dataset()
icpag = build_dataset.assign_links_to_datasets(icpag, extents)

# Model
modelpath = rf"{folder}/mobnet_v3_size{size}_tiles{tiles}_sample{sample}_{best_case_epoch}"
model = tf.keras.models.load_model(
           modelpath, compile=True
        )

# Run predictions
grid_preds_folder = f"{path_repo}/data/data_out/gridded_predictions/mobnet_v3_size{size}_tiles{tiles}_sample{sample}"
os.makedirs(grid_preds_folder, exist_ok=True)

for name, ds in datasets.items():
    df_preds = build_dataset.get_gridded_images_for_dataset(
        model=model, ds=ds, icpag=icpag, tiles=1, size=size, resizing_size=128, bias=2, sample=1, to8bit=True
    )
    df_preds.to_parquet(f"{grid_preds_folder}/{name}.parquet")

In [ ]:
m = df_preds.explore(column="predictions", cmap="Spectral",
                tiles="https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="ESRI",
                vmax=df_preds.predictions.quantile(.95), vmin=df_preds.predictions.quantile(.05)
)
folium.LayerControl().add_to(m)
m

In [ ]:
import build_dataset

build_dataset.get_random_images_for_link(
    ds, icpag, link, 2, 512, 128, 2, 5, to8bit=True
)